In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd

# Chemin vers votre WebDriver
DRIVER_PATH = r"C:\Users\Nabila Rouar\Downloads\chromedriver-win64\chromedriver.exe"

# URL de la page contenant la liste des champions
URL = "https://leagueoflegends.fandom.com/wiki/List_of_champions"

# Créer une instance du service
service = Service(DRIVER_PATH)

# Initialiser le navigateur
driver = webdriver.Chrome(service=service)
driver.get(URL)

# Attendre que la table des champions soit visible
try:
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.CLASS_NAME, "article-table"))
    )
    print("Table des champions trouvée et chargée avec succès.")
except Exception as e:
    print(f"Erreur : {e}")
    driver.quit()
    exit()

# Récupérer le HTML de la page des champions
html = driver.page_source

# Fermer le navigateur après récupération des données
driver.quit()

# Analyser le HTML avec BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Sélectionner la table des champions
table = soup.find("table", class_="article-table sticky-header sortable jquery-tablesorter")
champions = []

if table:
    rows = table.find("tbody").find_all("tr")
    
    # Récupérer les 5 premiers champions de la liste
    for row in rows[:5]:  # On limite à 5 premiers champions pour le test
        cells = row.find_all("td")
        if len(cells) > 0:
            champion_cell = cells[0]
            
            # Extraire le nom et le titre du champion
            name_div = champion_cell.find("span", class_="inline-image label-only champion-icon tooltips-init-complete")
            if name_div:
                name = name_div.get("data-champion", "Unknown")
                title = name_div.get_text(strip=True)
            
            # Récupérer le lien complet du champion
            link_tag = champion_cell.find("a", href=True)
            if link_tag:
                champion_link = "https://leagueoflegends.fandom.com" + link_tag['href']
            else:
                champion_link = "Unknown"

            # Récupérer l'image du champion
            img_tag = champion_cell.find("img")
            if img_tag:
                img_url = img_tag.get("data-src", "Unknown")
            else:
                img_url = "Unknown"

            # Ajouter les données de base aux champions
            champions.append({
                "Name": name,
                "Title": title,
                "Champion Link": champion_link,
                "Image URL": img_url,
                "Position": "Non trouvée",  # Valeur par défaut pour la position
                "Resource": "Non trouvée",   # Valeur par défaut pour la resource
                "Rangetype": "Non trouvée"  # Valeur par défaut pour rangetype
            })

# Initialiser le navigateur pour récupérer les détails de chaque champion
driver = webdriver.Chrome(service=service)

# Parcourir les champions pour obtenir la position et la ressource
for champion in champions:
    champion_url = champion["Champion Link"]
    driver.get(champion_url)

    # Attendre que les sections "position", "resource" et "rangetype" soient visibles
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ".pi-item[data-source='position']"))
        )
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ".pi-item[data-source='resource']"))
        )
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ".pi-item[data-source='rangetype']"))
        )
    except:
        print(f"Erreur : Impossible de trouver les sections pour {champion_url}")
        continue

    # Récupérer le HTML de la page du champion
    html_champion = driver.page_source
    soup_champion = BeautifulSoup(html_champion, 'html.parser')

    # Extraire la position
    position_section = soup_champion.find("div", class_="pi-item", attrs={"data-source": "position"})
    if position_section:
        position_span = position_section.find("span", class_="glossary tooltips-init-complete")
        if position_span:
            champion["Position"] = position_span.get_text(strip=True)

    # Extraire la resource
    resource_section = soup_champion.find("div", class_="pi-item", attrs={"data-source": "resource"})
    if resource_section:
        resource_span = resource_section.find("span", class_="glossary tooltips-init-complete")
        if resource_span:
            champion["Resource"] = resource_span.get_text(strip=True)

    # Extraire le rangetype
    rangetype_section = soup_champion.find("div", class_="pi-item", attrs={"data-source": "rangetype"})
    if rangetype_section:
        rangetype_span = rangetype_section.find("span", class_="glossary tooltips-init-complete")
        if rangetype_span:
            champion["Rangetype"] = rangetype_span.get_text(strip=True)

# Créer un DataFrame avec toutes les informations
df = pd.DataFrame(champions)

# Sauvegarder toutes les données dans un CSV
df.to_csv("champions_first_5_info.csv", index=False)
print("Toutes les données des 5 premiers champions ont été sauvegardées dans 'champions_first_5_info.csv'.")

# Fermer le navigateur à la fin
driver.quit()

import pandas as pd

# Lire les données du CSV généré précédemment
df = pd.read_csv("champions_first_5_info.csv")

# Définir le modèle HTML
html_template = """
<!DOCTYPE html>
<html lang="fr">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Champions de League of Legends</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            margin: 20px;
            background-color: #f4f4f9;
        }
        .container {
            display: flex;
            flex-wrap: wrap;
            gap: 20px;
        }
        .champion-card {
            width: 200px;
            border: 1px solid #ccc;
            border-radius: 10px;
            padding: 10px;
            background-color: #fff;
            box-shadow: 0 2px 10px rgba(0, 0, 0, 0.1);
        }
        .champion-card img {
            width: 100%;
            height: auto;
            border-radius: 5px;
        }
        .champion-card h3 {
            text-align: center;
            font-size: 18px;
            margin-top: 10px;
        }
        .champion-card p {
            font-size: 14px;
            color: #555;
            margin: 5px 0;
        }
        .champion-card .title {
            font-weight: bold;
        }
    </style>
</head>
<body>
    <h1>Liste des Champions de League of Legends</h1>
    <div class="container">
"""

# Ajouter les informations de chaque champion à la page
for index, row in df.iterrows():
    champion_card = f"""
    <div class="champion-card">
        <img src="{row['Image URL']}" alt="{row['Name']}">
        <h3>{row['Name']}</h3>
        <p><span class="title">Title:</span> {row['Title']}</p>
        <p><span class="title">Position:</span> {row['Position']}</p>
        <p><span class="title">Resource:</span> {row['Resource']}</p>
        <p><span class="title">Rangetype:</span> {row['Rangetype']}</p>
        <a href="{row['Champion Link']}" target="_blank">Voir le champion</a>
    </div>
    """

    html_template += champion_card

# Clôturer le HTML
html_template += """
    </div>
</body>
</html>
"""

# Sauvegarder le fichier HTML
with open("champions_page.html", "w", encoding="utf-8") as f:
    f.write(html_template)

print("Page web générée avec succès : champions_page.html")
